# Exploring eBay Car Sales

The aim of this project is to clean the data and analyze the included used car listings. You'll also become familiar with some of the unique benefits jupyter notebook provides for pandas.

The orriginally dataset we are working with is of Over 370000 used cars scraped with Scrapy from Ebay-Kleinanzeigen, you can find [here](https://data.world/data-society/used-cars-data).

The current dataset have some diffences:
- We sampled 50,000 data points from the full dataset, to ensure your code runs quickly in our hosted environment
- We dirtied the dataset a bit to more closely resemble what you would expect from a scraped dataset (the version uploaded to Kaggle was cleaned to be easier to work with)

## Exploring dataset
We'll start importing [numpy](https://numpy.org/) and [pandas](https://pandas.pydata.org/) to start work with the dataset.

In [1]:
import numpy as np
import pandas as pd

We'll use pandas [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html?highlight=read_csv#pandas-read-csv) function to read the dataset.

Note: We'll use `Latin-1` encoding for dataset.

In [2]:
autos = pd.read_csv('autos.csv', encoding='Latin-1')

We'll use [DataFrame.info()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html?highlight=info#pandas.DataFrame.info) and [DataFrame.head()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.head.html?highlight=head#pandas.DataFrame.head) functions to start exploring the dataset  structure and the content.

In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

- We can see that the columns `vehicleType`, `gearbox`, `model`, `fuelType` and `notRepairedDamage` contains null values.
- The column names use camelcase instead of Python's preferred snakecase, which means we can't just replace spaces with underscores.

In [4]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


We can observe in the content of the first 5 columns.
- All colums (`dateCrawled`, `dateCreated`, `lastSeen`) that contain a date are actually datetimes.
- In the `price` column the amounts are formatted as US dollar, for example `$1,000.00`.
- In the `odometer` column the quantities contain the subfix of the unit of measure (`km`) and contain commas. 

## Snakecase for columns name
As we saw earlier, the column names are in `camelcase`, the first thing we will do is cleaning them to `snakecase` (Python convention). For that we'll use the [DataFrame.rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html?highlight=rename#pandas.DataFrame.rename) (with `columns` param to specify new columns names, and `inplace=True` arg to make change in current DataFrame).


For some columns we'll make an special rename of column:
- `yearOfRegistration` to `registration_year`
- `monthOfRegistration` to `registration_month`
- `notRepairedDamage` to `unrepaired_damage`
- `dateCreated` to `ad_created`
- `nrOfPictures` to `nr_pictures`

In [5]:
# The key are the current colum name
# The value are the new column name.
columns = {
    'dateCrawled': 'date_crawled',
    'offerType': 'offer_type',
    'vehicleType': 'vehicle_type',
    'powerPS': 'power_ps',
    'fuelType': 'fuel_type',
    'postalCode': 'postal_code',
    'lastSeen': 'last_seen',
    'yearOfRegistration': 'registration_year',
    'monthOfRegistration': 'registration_month',
    'notRepairedDamage': 'unrepaired_damage',
    'dateCreated': 'ad_created',
    'nrOfPictures': 'nr_pictures'
}

In [6]:
autos.rename(columns=columns, inplace=True)

## Initial data clening
Now let's do some basic data exploration to determine what other cleaning tasks need to be done. Initially we will look for:

- Text columns where all or almost all values are the same. These can often be dropped as they don't have useful information for analysis.
- Examples of numeric data stored as text which can be cleaned and converted

We'll use [DataFrame.describe()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html?highlight=describe#pandas.DataFrame.describe) (with include='all' to get both categorical and numeric columns) to look at descriptive statistics for all columns.

In [7]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


We can see the columns `seller`,`offer_type` have mostly one value, that are candidate to be dropped.

In [13]:
autos.value_counts('seller')

seller
privat        49999
gewerblich        1
dtype: int64

In [14]:
autos.value_counts('offer_type')

offer_type
Angebot    49999
Gesuch         1
dtype: int64

Any examples of numeric data stored as text that needs to be cleaned. We saw the `price` and `odometer` columns are numeric values stored as text. We'll remove any non-numeric character and convert column to numeric `dtype`.

We found the format for `price` column are `$1,000.00`, we need to remove the `$` and `,` characters.
We'll going to use:
- [Series.str](https://pandas.pydata.org/docs/reference/api/pandas.Series.html) vectorized string function to replace the characters.
- [Series.astype()](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html) to cast to `int` dtype.


In [9]:
autos['price'] = autos['price'].str.replace('$', '').str.replace(',', '').astype(int)
autos.value_counts('price')

price
0           1421
500          781
1500         734
2500         643
1000         639
            ... 
8098           1
8099           1
8180           1
40990          1
99999999       1
Length: 2357, dtype: int64

We found the format for `odometer` column are `1,000km`, we need to remove the `km` and `,` characters. For this column we will also rename the column adding the suffix `_km` in order to not lose information of unit measure.


In [10]:
autos['odometer'] = autos['odometer'].str.replace('km', '').str.replace(',', '').astype(int)
autos.value_counts('odometer')

odometer
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
dtype: int64

In [11]:
autos.rename(columns={'odometer': 'odometer_km'}, inplace=True)
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_pictures', 'postal_code',
       'last_seen'],
      dtype='object')